<b>Telco Churn Classifcation Project</b>
<b>Final Report</b><br>
Stephanie Jones<br>
Codeup | Hopper, Data Science<br>
Monday, November 28, 2021

In [1]:
# # created a for loop that returned a list of our (non-numerical) object columns and stored in variable obj_cols

# # empty ilst that will store the columns that meet the below criteria in for loop
# object_cols = []

# for col in raw_data.columns:
#     if raw_data[col].dtype == 'O':
#         object_cols.append(col)

In [2]:
# # these (our object columns) are the columns we will need to map to numerical values or encode
# print(object_cols)
# print('\n')
# print(f'There are {len(object_cols)} object columns.')


# 01_acquire
To acquire the Telco data, I used the 'telco_churn' db in our mySQL server, and selected all columns from customers table `select * from customers`<br><br>
joining the following tables:
- contract_types using (contract_type_id) `join contract_types using (contract_type_id)`
- internet_service_types using (internet_service_type_id) `join internet_service_types using (internet_service_type_id)`
- payment_types using (payment_type_id)`join payment_types using (payment_type_id)`

In [3]:
import acquire # this is the file with all of the functions used to connect to db and acquire Telco data

# this will run the function that:
#     connects to the db using credentials from env file
#     runs the SQL query and writes a pandas DataFrame from the results
#     caches the DataFrame into a local telco_data.csv file if one does not already exist

raw_data = acquire.acquire_telco_data()

# 02_prepare

To prepare the Telco data for exploration and modeling, I used the raw_telco data and did the following:<br><br>
<b>Cleaning</b>
-         a. removed x11 rows where `total_charges` was an empty valuie
-         b. changed `total_charges` values from object to float dtype
-         c. created dummy variables for object columns:
                                                'churn', 'contract_type', 'dependents',
                                                'device_protection', 'gender', 'internet_service_type', 
                                                'multiple_lines', 'online_backup', 'online_security', 
                                                'paperless_billing', 'partner', 'payment_type', 'phone_service', 
                                                'streaming_movies', 'streaming_tv', 'tech_support'
-         d. concatenated dummy columns with the original df
-         e. dropped unnecessary columns and customer_id column, which is not needed for exploration and modeling:
    - e-1. dummy columns w/boolean values:
                                                'churn_No', 'dependents_No', 'gender_Female', 'paperless_billing_No',
                                                'partner_No', 'phone_service_No'
    - e-2. dropped unneeded customer_id column
    - e-3.dropped redundant columns:
                                                'contract_type_id', 'internet_service_type_id', 'payment_type_id'

-         f. renamed columns to be shorter and more understable:
                                                'churn_Yes': 'churn', 'contract_type_Month-to-month': 'contract_mtm',
                                                'contract_type_One year': 'contract_one_yr', 'contract_type_Two year':
                                                'contract_two_yr', 'dependents_Yes': 'dependents', 'gender_Male': 
                                                'gender_m', 'internet_service_type_DSL': 'internet_dsl', 
                                                'internet_service_type_Fiber optic': 'internet_fiber', 
                                                'internet_service_type_None': 'internet_none', 'paperless_billing_Yes':
                                                'paperless_bill', 'partner_Yes': 'partner', 
                                                'payment_type_Bank transfer (automatic)': 'pay_xfer_auto', 
                                                'payment_type_Credit card (automatic)': 'pay_credit_auto', 
                                                'payment_type_Electronic check': 'pay_echeck', 
                                                'payment_type_Mailed check':'pay_mail', 'phone_service_Yes': 
                                                'phone_service'
-         g. lowercased column names


<b>Splitting</b>
-         a. split the cleaned telco data, first into `train_and_validate` (80%) and `test` (20%)
-         b. splt the `train_and_validate` into `train` (70% of the 80%, <i>56% of the total</i>) and `validate` (30% of the 80%, <i>24% of the total</i>)

In [5]:
import prepare # this is the file with all of the functions used to clean and split the raw Telco data

train, validate, test = prepare.clean_split_telco_data(raw_data)

In [36]:
print(f'Train: {train.shape}') 
print(f'Validate: {validate.shape}') 
print(f'Test: {test.shape}') 

Train: (3937, 41)
Validate: (1688, 41)
Test: (1407, 41)
